In [6]:
import logging
logging.getLogger().setLevel(logging.WARNING)

from konlpy.corpus import kobill, kolaw
from konlpy.tag import Twitter
twitter = Twitter()

stopwords = "수 년 등 및 몇 중 네이버 뉴스".split()
stopwords += "공급 설치 조성 운영 실행 설립 확대 제공 실시 지원 검토 육성 추진 유치 강화 개선 구축 마련 확충 실시 개선 해소".split()
stopwords = set(stopwords)

pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True) if p[0] not in stopwords and p[1] in ('Noun', 'Verb', 'Alpha', 'Adjective')]

In [7]:
twitter.pos('청각장애인과 함께 CCTV로 만드는 안전한 서울', stem=True, norm=True)

[('청각장애', 'Noun'),
 ('인과', 'Noun'),
 ('함께', 'Adverb'),
 ('CCTV', 'Alpha'),
 ('로', 'Noun'),
 ('만들다', 'Verb'),
 ('안전하다', 'Adjective'),
 ('서울', 'Noun')]

In [8]:
from konlpy.tag import Hannanum
hannanum = Hannanum()

twitter.pos('창출을 춘다', stem=True)
#hannanum.pos('창출을 춘다')

[('창', 'Noun'), ('추다', 'Verb'), ('추다', 'Verb')]

In [9]:
%%time
texts_kobill = [pos(kobill.open(i).read()) for i in kobill.fileids()]
texts_kolaw = [pos(kolaw.open(i).read()) for i in kolaw.fileids()]

CPU times: user 1.41 s, sys: 21.4 ms, total: 1.44 s
Wall time: 1.71 s


In [20]:
%%time
import json

texts = []

for fname in ('jisa_tagged.json', 'mayor_tagged.json',):
    with open(fname) as data_file:    
        data = json.load(data_file)
        for voting_district in data:
            if voting_district['city'] != 'seoul' or voting_district['district'] > 28:
                continue
            promises = voting_district['promises']
            for p in promises:
                texts.append((p['title'], pos(p['title']), (voting_district['city'], voting_district['district'])))

texts_promises = [text[1] for text in texts]
print(len(texts_promises))

with open('test-data/promise-titles.json', 'w') as f:
    json.dump([text[0] for text in texts], f)

250
CPU times: user 393 ms, sys: 352 ms, total: 745 ms
Wall time: 1.3 s


In [21]:
from gensim import corpora, models
# Create dictionary
corpus = texts_kobill + texts_kolaw + texts_promises
#corpus = texts_promises
logging.info('Dictionary from %d documents' % len(corpus))
dictionary_ko = corpora.Dictionary(corpus)
# Filter terms that occur in more than 2% of the documents
dictionary_ko.filter_extremes(no_below=0, no_above=0.02)
# Save for later use
dictionary_ko.save('ko.dict')

In [22]:
%%time
from gensim import similarities

num_topics = 30

# Generate Lsi model for promise text corpus
corpus = [dictionary_ko.doc2bow(text) for text in texts_promises]
# Train Lsi model
lsi = models.LsiModel(corpus, id2word=dictionary_ko, num_topics=num_topics)
lsi.save('lsi.model')
# Transform corpus to LSI space and index it
lsi_corpus = lsi[corpus]
index = similarities.MatrixSimilarity(lsi_corpus)
index.save('lsi.index')

CPU times: user 184 ms, sys: 67.8 ms, total: 252 ms
Wall time: 365 ms


In [44]:
def find_similar(tagged_text, similarity_threshold=None):
    # Convert text to bag of words
    vec_bow = dictionary_ko.doc2bow(tagged_text)
    # Convert the query to LSI space
    vec_lsi = lsi[vec_bow]
    # Perform a similarity query against the corpus
    sims = enumerate(index[vec_lsi])
    if similarity_threshold:
        sims = filter(lambda item: item[1] >= similarity_threshold, sims)
    # Return the similarities sorted by descending score
    return sorted(sims, key=lambda item: -item[1])
    
# Try to match a few existing documents

for text in texts[100:101]:
    sims = find_similar(text[1], similarity_threshold=0.75)
    print("Most similar to %s (%s)" % (text[0], ' '.join(text[1])))
    for key, score in sims[1:]:
        key = int(key)
        print('%.2f - %s' % (score, texts[key][0]))


Most similar to 동북아 대도시 대기환경협의체 구성 및 대기개선 노하우 공유 (동북아/Noun 대도시/Noun 대기/Noun 환경/Noun 협의/Noun 체/Noun 구성/Noun 대기/Noun 노하우/Noun 공유/Noun)
0.99 - 교육지원협의체 구성
0.98 - 4) 복지 네트워크 구성 및 사회적 연대틀 강화
0.96 - 지방의회 산하 생활안정센터 구성
0.96 - 마포구 분양가 자문위원회 구성
0.96 - 성북구 생활구정위원회 구성·운영
0.96 - 주민자치위원회 구성의 다양화
0.95 - 생활구정 혁신본부 구성
0.95 - 도시재생본부 설립 및 도시재생자문위원회 운영
0.95 - 예술위원회 구성
0.94 - 각종 위원회 운영상황 상시공개 추진
0.94 - 구정 주민자치위원회 신설 및 동 자치위원회 활성화
0.94 - 종로비전위원회 설치
0.94 - 인사위원회 주민참여 확대
0.94 - 각종 위원회 주민참여 확대
0.94 - 구정운영위원회 설치 운영
0.94 - 다문화 가정 지원을 위한 전담팀 구성 · 운영
0.94 - 개방형 감사관 및 감사위원회 설치
0.94 - 4) 각급 위원회 활성화 및 전문가의 구정참여 실질화
0.94 - 위원회 회의자료 사전공개 및 결정 공개
0.93 - 주민참여형 지역개발위원회 상설 기구화
0.93 - 교육지원본부 신설
0.92 - 동대문구 일자리창출정책협의회 구성
0.92 - ∘한국문화예술위원회와 협력
0.91 - 인사위원회 공정성 확립
0.90 - 주민자치위원 공개 모집
0.87 - 구정 조직 재진단 및 쇄신
0.86 - 깨끗하고 투명한 청정구정 구현
0.85 - 동 자치회관 프로그램 개선
0.85 - 주민과만남 정례화(자치회관 프로그램 운영확대)
0.84 - 동자치회관 문화교양강좌 활성화
0.84 - 구룡마을 문제 해결을 위한 전문가협의체 구성 및 합리적 방안 모색
0.83 - 전문가가 참여하는 구정평가단 운영
0.83 - 4) 시민단체 협력과 구정참여 확대
0.82 - 4) 동별 자치문화 한마당 개최
0

In [87]:
%%time
import numpy as np
# Find similarities of all documents with each other and see how many matches higher than x score there are
# Used to figure out a threshold that regards enough documents as matched
threshold = 0.72
counts = []
counts_self_no_match = []
for idx, text in enumerate(texts[:500]):
    similarities = np.array(find_similar(text[1]))
    # Remove self-similarity
    id_idx = np.where(similarities[:, 0]==idx)[0][0]
    similarities = np.delete(similarities, id_idx, axis=0)
    # Record when self wasn't the best match (it is usally the 2nd best in these cases)
    if id_idx != 0:
        counts_self_no_match.append((idx, id_idx))
    # Count number of docs with similarity >= threshold
    count = np.sum(similarities[:,1]>=threshold)
    counts.append(count)

counts = np.array(counts)
counts_self_no_match = np.array(counts_self_no_match)
print('Number of matches per document: max %d, min %d, mean %.2f, median %d' % (np.max(counts), np.min(counts), np.mean(counts), np.median(counts)))
print('%d documents didn\'t match with themself as their best match' % (len(counts_self_no_match)))
if len(counts_self_no_match):
    print('  but %d of them matched at pos 1.' % (np.sum(counts_self_no_match[:,1]==1)))

no_match_count = np.sum(counts==0)
print('%d (%d%%) with 0 matches' % (no_match_count, 100*no_match_count/len(counts)))

Number of matches per document: max 44, min 0, mean 17.69, median 19
9 documents didn't match with themself as their best match
  but 2 of them matched at pos 1.
29 (5%) with 0 matches
CPU times: user 1.23 s, sys: 18.8 ms, total: 1.25 s
Wall time: 1.26 s


In [73]:
print("Document with highest number of matches:")
print(texts[np.argmax(counts)])

Document with highest number of matches:
('창의어린이 공원 조성사업', ['창의/Noun', '어린이/Noun', '공원/Noun', '사업/Noun'], ('seoul', 8))


In [47]:
print("Unmatched documents:")
for idx in np.where(counts==0)[0]:
    print(texts[idx][0], texts[idx][1])

Unmatched documents:
개포모바일 융합클러스터 조성사업 추진 ['개포/Noun', '모바일/Noun', '융합/Noun', '클러스터/Noun', '사업/Noun']
시장 직속 일자리 위원회 신설 ['시장/Noun', '직속/Noun', '일자리/Noun', '위원회/Noun', '신설/Noun']
한양도성 세계유산 등재 ['한양도성/Noun', '세계/Noun', '유산/Noun', '등재/Noun']
잠자리, 먹거리, 일거리가 한곳에서 해결되도록 지원 ['잠자리/Noun', '먹거리/Noun', '일거리/Noun', '곳/Noun', '해결/Noun', '되다/Verb']
불공정피해 상담센터 기능 확대 ['불공정/Noun', '피해/Noun', '상담/Noun', '센터/Noun', '기능/Noun']
서울시네마테크’ 건립 ['서울/Noun', '시네마테크/Noun', '건립/Noun']
지하철역 엘리베이터 설치 지속추진, 공공시설 장애인 동선 확보 ['지하철역/Noun', '엘리베이터/Noun', '지속/Noun', '공공시설/Noun', '장애인/Noun', '동선/Noun', '확보/Noun']
수도권연계 광역도시철도 신설 및 제3기 도시철도 조기 추진 ['수도/Noun', '권연/Noun', '광역/Noun', '도시철도/Noun', '신설/Noun', '제/Noun', '기/Noun', '도시철도/Noun', '조기/Noun']
서울 선데이파크에서 놀면서 살빼요 ['서울/Noun', '선데이/Noun', '파크/Noun', '놀다/Verb', '살빼다/Adjective']
사고위치 번호판 운영 ['사고/Noun', '위치/Noun', '번호판/Noun']
글로벌 의료관광 육성 ['글로벌/Noun', '의료관광/Noun']
금융기관 본점 등 건실한 기업 유치 ['금융기관/Noun', '본점/Noun', '건실/Noun', '기업/Noun']
자원봉사와 노인일자리 등 생산적인 노후생활 활성화 ['자원봉사/Noun', '노인/Noun', 

In [74]:
# Try some queries that don't match existing documents

def match(title):
    tags = pos(title)
    print("Most similar to %s (%s)" % (title, " ".join(tags)))
    threshold = 0.75
    for posi, item in enumerate(find_similar(tags, threshold)):
        key = int(item[0])
        score = item[1]
        print('%2d. %.2f - %s - %s %d' % (posi+1, score, texts[key][0], texts[key][2][0], texts[key][2][1]))

match('임대주택')
match(' 임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토')
match('지난해 저소득층 근로소득 감소에…소득 불평등도 악화 : 네이버 뉴스')
match('노후자금 털어 집에 투자? 전세금, 집값 용도 퇴직연금 중도인출 급증 : 네이버 뉴스')

text = "대학 등록금 동결 속 외국인 유학생만 '봉' 서울 주요 대학들이 외국인 유학생이 내야 할 등록금을 잇달아 올리려 하는 것으로 드러났다. 내국인 대학생에 대한 ‘등록금 동결’과 ‘입학금 폐지’로 줄어든 수입을 외국인 유학생에게서 더 받아내 만회하겠다는 것이다."
match(text)

Most similar to 임대주택 (임대주택/Noun)
 1. 0.97 - 장기 공공임대주택 2500호 공급 -> 장기 공공임대주택 2,400호 공급 - seoul 16
 2. 0.96 - 8) 대학생 임대주택 공급 지원 - seoul 17
 3. 0.95 - 태양광 설치비’ 장기 저금리 융자지원 - seoul 0
 4. 0.93 - 공공청사 태양광발전소 건립 확대 - seoul 2
 5. 0.93 - 공공보육시설 확충 - seoul 1
 6. 0.93 - 스마트형 공공 쓰레기통, ‘진격의 스마통’ - seoul 0
 7. 0.93 - 공공청사 장소대관 활성화 - seoul 2
 8. 0.92 - 공공·작은도서관 지속적 확충 및 운영 개선 - seoul 0
 9. 0.91 - 맞벌이 부부를 위한 역세권 공공보육시설 설치 - seoul 13
10. 0.90 - 폐기물 수거업무의 공공기능 확대 - seoul 26
11. 0.89 - 공공 노인요양원 30개소 설치 - seoul 0
12. 0.89 - 공공부문 일자리 종합대책 수립 - seoul 13
13. 0.87 - 대학생등록금 50% 무이자지원(구립대학생 공공기숙사 건립) -> 대학생 등록금 지원 장학사업 - seoul 26
14. 0.86 - 신축공공건물 옥상공원화 및 에너지효율1등급 건축물 건설 - seoul 26
15. 0.85 - 365일 24시간 운영 공공보육시설 시범 설치 - seoul 1
16. 0.85 - 서울시정 대학생 인턴십 프로그램 - seoul 0
17. 0.83 - 세대융합(화합)형 공공주택 공급 - seoul 0
18. 0.83 - 공공도서관 연계 활성화 - seoul 2
19. 0.82 - 용산구 장학기금 운영 - seoul 24
20. 0.81 - 공공부문 채용시 전체의 3%는 은퇴자, 노인 고용 의무화 추진 - seoul 16
Most similar to  임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토 (임대주택/Noun 등록/Noun 하다/Verb 세금/No

In [75]:
corpora.MmCorpus.serialize('lsi.mm', lsi_corpus)

In [76]:
%%time
import numpy as np
from sklearn.manifold import TSNE
from gensim import matutils

lsi_corpus = corpora.MmCorpus('lsi.mm')
tsne = TSNE(n_components=2, init='pca', random_state=0, metric='cosine', angle=.99)
X = matutils.corpus2dense(lsi_corpus, num_terms=lsi_corpus.num_terms)
X_data = np.asarray(X).astype('float64')
X_tsne = tsne.fit_transform(X_data.T)

CPU times: user 23.6 s, sys: 3.51 s, total: 27.1 s
Wall time: 30.1 s


In [81]:
# Only use topics with high enough propability
threshold = 0.1
_idx = np.amax(X, axis=0) > threshold
X_ = X[:, _idx]
X_tsne_ = X_tsne[_idx, :]

In [82]:
#X_tsne[_idx, 0].shape
X_.shape

(30, 972)

In [83]:
import matplotlib
matplotlib.rc('font', family='NanumGothic')

from matplotlib import pyplot as plt
top_topics = np.argmax(X_, axis=0)

topics_words = lsi.show_topics(num_words=5, formatted=False)
topic_summaries = [' '.join([word[0].split('/')[0] for word in topic]) for topic_no, topic in topics_words]

plt.figure(figsize=(15,10))
plt.scatter(X_tsne_[:, 0], X_tsne_[:, 1], c=top_topics, cmap=plt.cm.get_cmap("jet", 50))

plt.colorbar(ticks=range(50))
plt.clim(-0.5, 9.5)

# find center of topic clusters
topic_coord = np.empty((len(topic_summaries), 2)) * np.nan
for topic_num in range(len(topic_summaries)):
    if not np.isnan(topic_coord).any():
        break
    docs_in_topics = (top_topics==topic_num)
    if not docs_in_topics.any():
        # no documents were assigned to this topic
        continue
    #topic_coord[topic_num] = X_tsne_[top_topics==topic_num].mean(axis=0)
    topic_coord[topic_num] = X_tsne_[np.argmax(top_topics==topic_num)]

# plot topic summaries
for i in range(len(topic_summaries)):
    if np.isnan(topic_coord[i]).any():
        continue
    plt.text(topic_coord[i, 0], topic_coord[i, 1], topic_summaries[i])


#plt.show()
plt.savefig('tsne.png')

In [84]:
X_.shape

(30, 972)

In [85]:
import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import output_file, save
from bokeh.models import HoverTool
from random import shuffle

colors = ["#E495A5", "#E3979D", "#E19895", "#DE9A8E", "#DB9C86", "#D89F7F", "#D3A178", "#CEA372", "#C9A66D", "#C3A869", "#BDAB66", "#B6AD65", "#AFAF64", "#A7B166", "#9EB368", "#96B56C", "#8DB771", "#83B977", "#79BA7E", "#6FBB84", "#65BC8C", "#5ABD93", "#50BE9B", "#46BEA2", "#3FBEAA", "#39BEB1", "#37BDB8", "#3ABCBF", "#41BBC5", "#4ABACB", "#55B8D0", "#61B6D5", "#6DB4D9", "#79B1DC", "#85AEDF", "#91ACE1", "#9CA9E2", "#A6A6E2", "#B0A3E1", "#B9A0E0", "#C29DDE", "#C99ADB", "#D098D7", "#D596D3", "#DA95CD", "#DE94C8", "#E093C1", "#E393BB", "#E493B4", "#E494AC"]
shuffle(colors)
colormap = np.array(colors)
title = 'Promises LSI vizualization'
num_example = len(X_)

plot = bp.figure(plot_width=1200, plot_height=700,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     min_border=1)

plot.scatter(x='x', y='y',
                 color='color',
                 source=bp.ColumnDataSource({
                   "x": X_tsne_[:, 0], "y": X_tsne_[:, 1],
                   "content": np.array(['%s %s %d' % (text[0], text[2][0], text[2][1]) for text in texts])[_idx],
                   "topic_key": top_topics,
                   "topic_summary": [topic_summaries[i] for i in top_topics],
                   "color": colormap[top_topics]
                   }))

for i in range(len(topic_summaries)):
    if np.isnan(topic_coord[i]).any():
        continue
    plot.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])

    
hover = plot.select(dict(type=HoverTool))
hover.tooltips = {"content": "@content - topic: @topic_key @topic_summary"}
output_file('{}.html'.format(title))
save(plot)


'/Users/paul/Projects/kixlab/xproject/xproj-utils/promise-match/Promises LSI vizualization.html'